In [ ]:
!pip install geopy tqdm requests pandas

In [ ]:
import requests
import pandas as pd
from geopy.geocoders import Nominatim
from tqdm.notebook import tqdm
import time

In [ ]:
from google.colab import files

In [ ]:
print("📂 Please upload your base CSV (with State, District, Crop, etc.)")
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
df = pd.read_csv(file_name)
print(f"✅ Loaded dataset with {len(df)} rows")

📂 Please upload your base CSV (with State, District, Crop, etc.)


Saving APY.csv to APY.csv
✅ Loaded dataset with 345336 rows


In [ ]:
df.head()

,State,District,Crop,Crop_Year,Season,Area,Production,Yield
0,Andaman and Nicobar Island,NICOBARS,Arecanut,2007,Kharif,2439.6,3415.0,1.40
1,Andaman and Nicobar Island,NICOBARS,Arecanut,2007,Rabi,1626.4,2277.0,1.40
2,Andaman and Nicobar Island,NICOBARS,Arecanut,2008,Autumn,4147.0,3060.0,0.74
3,Andaman and Nicobar Island,NICOBARS,Arecanut,2008,Summer,4147.0,2660.0,0.64
4,Andaman and Nicobar Island,NICOBARS,Arecanut,2009,Autumn,4153.0,3120.0,0.75


In [ ]:
print(df.columns.tolist())
df.columns = df.columns.str.strip()
print(df.columns.tolist())

['State', 'District ', 'Crop', 'Crop_Year', 'Season', 'Area ', 'Production', 'Yield']
['State', 'District', 'Crop', 'Crop_Year', 'Season', 'Area', 'Production', 'Yield']


In [ ]:
# STEP 2: Upload the Kaggle district coordinates dataset
print("📂 Upload district coordinates dataset (with Latitude, Longitude)")
uploaded = files.upload()
coord_file = list(uploaded.keys())[0]
coords_df = pd.read_csv(coord_file, encoding='latin-1')

📂 Upload district coordinates dataset (with Latitude, Longitude)


Saving Ind_adm2_Points.csv to Ind_adm2_Points (1).csv


In [ ]:
coords_df.head()

,ÿCountry,District,Ind_adm2_ID,point_order,State,sub_polygon_id,Latitude,Longitude,Number of Records
0,India,Andaman Islands,0,326,Andaman and Nicobar,28,11.5957,92.5322,1
1,India,Andaman Islands,0,327,Andaman and Nicobar,28,11.5963,92.5325,1
2,India,Andaman Islands,0,328,Andaman and Nicobar,28,11.5969,92.5327,1
3,India,Andaman Islands,0,329,Andaman and Nicobar,28,11.5974,92.5329,1
4,India,Andaman Islands,0,330,Andaman and Nicobar,28,11.5980,92.5331,1


In [ ]:
# Keep only relevant columns
coords_df = coords_df[["State", "District", "Latitude", "Longitude"]]

In [ ]:
coords_df.head()

,State,District,Latitude,Longitude
0,Andaman and Nicobar,Andaman Islands,11.5957,92.5322
1,Andaman and Nicobar,Andaman Islands,11.5963,92.5325
2,Andaman and Nicobar,Andaman Islands,11.5969,92.5327
3,Andaman and Nicobar,Andaman Islands,11.5974,92.5329
4,Andaman and Nicobar,Andaman Islands,11.5980,92.5331


In [ ]:
# STEP 3: Merge both datasets
df = df.merge(coords_df, on=["State", "District"], how="left")
print(f"✅ Merged successfully — now have {len(df)} records")

✅ Merged successfully — now have 345336 records


In [ ]:
df.head()

,State,District,Crop,Crop_Year,Season,Area,Production,Yield,Latitude,Longitude
0,Andaman and Nicobar Island,NICOBARS,Arecanut,2007,Kharif,2439.6,3415.0,1.40,NaN,NaN
1,Andaman and Nicobar Island,NICOBARS,Arecanut,2007,Rabi,1626.4,2277.0,1.40,NaN,NaN
2,Andaman and Nicobar Island,NICOBARS,Arecanut,2008,Autumn,4147.0,3060.0,0.74,NaN,NaN
3,Andaman and Nicobar Island,NICOBARS,Arecanut,2008,Summer,4147.0,2660.0,0.64,NaN,NaN
4,Andaman and Nicobar Island,NICOBARS,Arecanut,2009,Autumn,4153.0,3120.0,0.75,NaN,NaN


In [ ]:
# =============== STEP 2: Geocode to get latitude & longitude ===============
geolocator = Nominatim(user_agent="geoapi", timeout=10)

In [ ]:
def get_lat_lon(State, District):
    """Get coordinates for a district and state in India"""
    try:
        location = geolocator.geocode(f"{District}, {State}, India")
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except Exception as e:
        return None, None

# Use tqdm for progress bar
tqdm.pandas()



In [ ]:
df["Latitude"], df["Longitude"] = zip(*df.progress_apply(lambda x: get_lat_lon(x["State"], x["District"]), axis=1))

  0%|          | 0/345336 [00:00<?, ?it/s]

In [ ]:
# =============== STEP 3: Fetch NASA POWER Data (Rainfall & Temp) ===============
def get_nasa_power_data(lat, lon):
    """Fetch average annual temperature (°C) and rainfall (mm)"""
    if pd.isna(lat) or pd.isna(lon):
        return None, None
    try:
        url = (
            f"https://power.larc.nasa.gov/api/temporal/climatology/point?"
            f"parameters=T2M,PRECTOT&community=AG&longitude={lon}&latitude={lat}&format=JSON"
        )
        r = requests.get(url).json()
        temp = r["properties"]["parameter"]["T2M"]["ANN"]
        rain = r["properties"]["parameter"]["PRECTOT"]["ANN"]
        return temp, rain
    except Exception:
        return None, None

df["Temp_avg"], df["Rainfall_mm"] = zip(*df.progress_apply(
    lambda x: get_nasa_power_data(x["Latitude"], x["Longitude"]), axis=1
))

In [ ]:
!pip install tqdm requests pandas numpy

import pandas as pd
import numpy as np
import requests
from tqdm import tqdm

# ===============================
# STEP 1 — Load Datasets
# ===============================
crop_df = pd.read_csv("APY.csv")
coords_df = pd.read_csv("district_coords.csv")

# Clean names for matching
def clean_name(x):
    return str(x).strip().lower().replace(" ", "")

crop_df["State_clean"] = crop_df["State"].apply(clean_name)
crop_df["District_clean"] = crop_df["District"].apply(clean_name)
coords_df["State_clean"] = coords_df["State"].apply(clean_name)
coords_df["District_clean"] = coords_df["District"].apply(clean_name)

# Merge coordinates
merged_df = pd.merge(
    crop_df,
    coords_df[["State_clean", "District_clean", "Latitude", "Longitude"]],
    on=["State_clean", "District_clean"],
    how="left"
)

print("✅ Coordinates merged:", merged_df[["State", "District", "Latitude", "Longitude"]].head())

# ===============================
# STEP 2 — Helper Functions
# ===============================

def get_weather_data(lat, lon, year):
    """
    Fetches mean rainfall and temperature for the given year using Open-Meteo API.
    """
    try:
        url = (
            f"https://archive-api.open-meteo.com/v1/era5?"
            f"latitude={lat}&longitude={lon}&start_date={year}-01-01&end_date={year}-12-31"
            f"&daily=temperature_2m_max,temperature_2m_min,precipitation_sum&timezone=Asia/Kolkata"
        )
        r = requests.get(url, timeout=10)
        data = r.json()
        temps = (np.array(data["daily"]["temperature_2m_max"]) + np.array(data["daily"]["temperature_2m_min"])) / 2
        rain = np.array(data["daily"]["precipitation_sum"])
        return temps.mean(), rain.mean()
    except Exception:
        return np.nan, np.nan


def get_soil_ph(lat, lon):
    """
    Fetches soil pH from SoilGrids API.
    """
    try:
        url = f"https://rest.isric.org/soilgrids/v2.0/properties/query?lon={lon}&lat={lat}&property=phh2o"
        r = requests.get(url, timeout=10)
        data = r.json()
        ph = data["properties"]["layers"][0]["depths"][0]["values"]["mean"] / 10  # Convert from *10 scale
        return ph
    except Exception:
        return np.nan


def get_ndvi(lat, lon, year):
    """
    Approximates mean NDVI using NASA POWER vegetation index proxy.
    (Simplified due to lack of direct free NDVI API)
    """
    try:
        url = (
            f"https://power.larc.nasa.gov/api/temporal/daily/point?parameters=NDVI"
            f"&community=AG&longitude={lon}&latitude={lat}&start={year}0101&end={year}1231&format=JSON"
        )
        r = requests.get(url, timeout=10)
        data = r.json()
        ndvi_values = list(data["properties"]["parameter"]["NDVI"].values())
        return np.mean(ndvi_values)
    except Exception:
        return np.nan

# ===============================
# STEP 3 — Enrich Dataset
# ===============================
ndvi_list, rainfall_list, temp_list, soilph_list = [], [], [], []

for _, row in tqdm(merged_df.iterrows(), total=len(merged_df)):
    lat, lon, year = row["Latitude"], row["Longitude"], int(row["Crop_Year"])

    if pd.isna(lat) or pd.isna(lon):
        ndvi_list.append(np.nan)
        rainfall_list.append(np.nan)
        temp_list.append(np.nan)
        soilph_list.append(np.nan)
        continue

    temp_avg, rainfall_mm = get_weather_data(lat, lon, year)
    soil_ph = get_soil_ph(lat, lon)
    ndvi_mean = get_ndvi(lat, lon, year)

    ndvi_list.append(ndvi_mean)
    rainfall_list.append(rainfall_mm)
    temp_list.append(temp_avg)
    soilph_list.append(soil_ph)

# Add to DataFrame
merged_df["NDVI_mean"] = ndvi_list
merged_df["Rainfall_mm"] = rainfall_list
merged_df["Temp_avg"] = temp_list
merged_df["Soil_pH"] = soilph_list

# ===============================
# STEP 4 — Save Final Dataset
# ===============================
merged_df.to_csv("enriched_crop_yield_dataset.csv", index=False)
print("✅ Enriched dataset saved successfully!")

# Check few rows
merged_df.head()


In [ ]:
# ==============================================
# Crop Yield Prediction Model Comparison
# ==============================================
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

# ==============================================
# Load & Prepare Data
# ==============================================
df = pd.read_csv("APY.csv")

df.columns = df.columns.str.strip()
print(df.columns.tolist())

# Encode categorical features
for col in ["State", "District", "Season", "Crop"]:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))

X = df[["State", "District", "Crop", "Crop_Year", "Season", "Area"]]
y = df["Yield"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ==============================================
# Define Models
# ==============================================
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(max_depth=10, random_state=42),
    "Random Forest": RandomForestRegressor(n_estimators=200, random_state=42),
    "XGBoost": XGBRegressor(n_estimators=300, learning_rate=0.1, random_state=42),
    "Neural Network": MLPRegressor(hidden_layer_sizes=(64, 32), max_iter=500, random_state=42)
}

# ==============================================
# Train & Evaluate Each Model
# ==============================================
results = []
for name, model in models.items():
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, pred)
    r2 = r2_score(y_test, pred)
    results.append((name, mae, r2))
    print(f"{name} -> MAE: {mae:.2f}, R2: {r2:.3f}")

# ==============================================
# Compare Results
# ==============================================
results_df = pd.DataFrame(results, columns=["Model", "MAE", "R2"])
print("\nModel Comparison:\n")
print(results_df.sort_values(by="R2", ascending=False))

# ==============================================
# Optional Physics-Informed Correction
# ==============================================
best_model = RandomForestRegressor(n_estimators=200, random_state=42)
best_model.fit(X_train, y_train)
pred = best_model.predict(X_test)

# Simulated physics-based inputs
temp_avg = np.random.uniform(20, 40, size=len(X_test))
ndvi_mean = np.random.uniform(0.2, 0.8, size=len(X_test))

def apply_physics_constraints(pred, temp, ndvi):
    correction = np.where(temp > 35, pred * 0.9, pred)
    correction = np.where(ndvi < 0.4, correction * 0.85, correction)
    return correction

corrected_pred = apply_physics_constraints(pred, temp_avg, ndvi_mean)
print("\nPhysics-Informed R2:", r2_score(y_test, corrected_pred))


['State', 'District', 'Crop', 'Crop_Year', 'Season', 'Area', 'Production', 'Yield']
Linear Regression -> MAE: 203.01, R2: 0.028
Decision Tree -> MAE: 19.85, R2: 0.888
Random Forest -> MAE: 14.83, R2: 0.915
XGBoost -> MAE: 20.36, R2: 0.913
Neural Network -> MAE: 88.57, R2: 0.434

Model Comparison:

               Model         MAE        R2
2      Random Forest   14.833752  0.915285
3            XGBoost   20.364981  0.913131
1      Decision Tree   19.848712  0.888203
4     Neural Network   88.565373  0.434318
0  Linear Regression  203.007006  0.028406

Physics-Informed R2: 0.90103330356107


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# ==============================================
# Crop Yield Prediction Model Comparison
# ==============================================
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor

# ==============================================
# Load & Prepare Data
# ==============================================
df = pd.read_csv("APY.csv")

df.columns = df.columns.str.strip()
print(df.columns.tolist())

# Encode categorical features
for col in ["State", "District", "Season", "Crop"]:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))

X = df[["State", "District", "Crop", "Crop_Year", "Season", "Area"]]
y = df["Yield"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ==============================================
# Define Models
# ==============================================
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(max_depth=10, random_state=42),
    "Random Forest": RandomForestRegressor(n_estimators=200, random_state=42),
    "XGBoost": XGBRegressor(n_estimators=300, learning_rate=0.1, random_state=42),
    "Neural Network": MLPRegressor(hidden_layer_sizes=(64, 32), max_iter=500, random_state=42)
}

# ==============================================
# Train & Evaluate Each Model
# ==============================================
results = []
for name, model in models.items():
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, pred)
    r2 = r2_score(y_test, pred)
    results.append((name, mae, r2))
    print(f"{name} -> MAE: {mae:.2f}, R2: {r2:.3f}")

# ==============================================
# Compare Results
# ==============================================
results_df = pd.DataFrame(results, columns=["Model", "MAE", "R2"])
print("\nModel Comparison:\n")
print(results_df.sort_values(by="R2", ascending=False))

# ==============================================
# Optional Physics-Informed Correction
# ==============================================
best_model = RandomForestRegressor(n_estimators=200, random_state=42)
best_model.fit(X_train, y_train)
pred = best_model.predict(X_test)

# Simulated physics-based inputs
temp_avg = np.random.uniform(20, 40, size=len(X_test))
ndvi_mean = np.random.uniform(0.2, 0.8, size=len(X_test))

def apply_physics_constraints(pred, temp, ndvi):
    correction = np.where(temp > 35, pred * 0.9, pred)
    correction = np.where(ndvi < 0.4, correction * 0.85, correction)
    return correction

corrected_pred = apply_physics_constraints(pred, temp_avg, ndvi_mean)
print("\nPhysics-Informed R2:", r2_score(y_test, corrected_pred))


['State', 'District', 'Crop', 'Crop_Year', 'Season', 'Area', 'Production', 'Yield']
Linear Regression -> MAE: 203.01, R2: 0.028
Decision Tree -> MAE: 19.85, R2: 0.888
Random Forest -> MAE: 14.83, R2: 0.915
XGBoost -> MAE: 20.36, R2: 0.913
Neural Network -> MAE: 88.57, R2: 0.434

Model Comparison:

               Model         MAE        R2
2      Random Forest   14.833752  0.915285
3            XGBoost   20.364981  0.913131
1      Decision Tree   19.848712  0.888203
4     Neural Network   88.565373  0.434318
0  Linear Regression  203.007006  0.028406

Physics-Informed R2: 0.90103330356107
